# Testing merges 

In [1]:
import numpy as np
import pandas as pd
import math
from siuba import *
from calitp import *
import intake
import data_prep
import altair as alt
from shared_utils import altair_utils 
pd.options.display.max_rows = 250

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.


In [2]:
df_5311 = data_prep.load_grantprojects()
vehicles = data_prep.load_cleaned_vehiclesdata()
organizations = data_prep.load_cleaned_organizations_data()
GTFS = data_prep.GTFS()

/home/jovyan/data-analyses/5311/data_prep.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/jovyan/data-analyses/5311/data_prep.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/jovyan/data-analyses/5311/data_prep.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ve

# Merge NTD with GTFS  --> vehicle_gtfs
<b> NOTE</b>:
I had to manually add NTD ID to the following agencies in the original CSV file called "cleaned organizations.csv"

* Butte County Association of Governments	90208
* City of Dixon	9402-91041
* City of Fairfield	90092
* City of Ridgecrest	9R02-91006
* City of Wasco	9R02-99426
* Glenn County	9R02-91088
* Mariposa County	9R02-91082
* Modoc Transportation Authority 	9R02-91008
* Palo Verde Valley Transit Agency	9R02-99454
* San Benito County Local Transportation Authority	9R02-91009
* San Joaquin Regional Transit District	90012
* Tuolumne County Transit Agency	9402-035


In [4]:
#trying the iloc way 
#organizations.loc[(organizations['name'] == 'Butte County Association of Governments'), "ntd_id"] = "90208"
#vehicles.loc[(vehicles['agency'] == 'Butte County Association of Governments'), "ntd_id"] = "90208"

#organizations.loc[(organizations['name'] == 'City of Fairfield'), "ntd_id"] = "90092"
#vehicles.loc[(vehicles['agency'] == 'City of Fairfield, California, dba: Fairfield and Suisun Transit'), "ntd_id"] = "90092"

In [5]:
#merging the 2 datasets together
vehicles_gtfs = pd.merge(vehicles, organizations,  how='left', on=['ntd_id'], indicator=True)

In [6]:
vehicle_agencies = vehicles_gtfs['agency'].drop_duplicates().tolist()

In [7]:
vehicles_gtfs['_merge'].value_counts()

both          192
left_only      26
right_only      0
Name: _merge, dtype: int64

In [8]:
vehicles_gtfs = vehicles_gtfs.drop(columns = ['_merge'])

# Merging -> Vehicle & GTFS with Black Cat.

In [9]:
#left merge, Black Cat on the left
Test1 = pd.merge(df_5311, vehicles_gtfs,  how='left', left_on=['organization_name'], right_on=['name'], indicator=True)

In [10]:
# Counting left only and both
Test1['_merge'].value_counts()

both          495
left_only     303
right_only      0
Name: _merge, dtype: int64

In [11]:
#finding the agencies that are in left only.
Left_only = Test1[(Test1._merge.str.contains("left_only", case= False))] 
Left_orgs = Left_only['organization_name'].drop_duplicates().tolist()

In [12]:
# Filter out left only matches
m2 = Test1[~Test1.organization_name.isin(Left_orgs)]

In [13]:
#check that filter worked ok
m2.shape

(495, 44)

### Failed Matches, crosswalk manually

In [14]:
#making a data frame with only failed merges out out of original Black Cat
fail = df_5311[df_5311.organization_name.isin(Left_orgs)]

In [15]:
fail.shape

(303, 15)

In [16]:
#crosswalk dictionary for replacing organization name
crosswalk = {'City of Chowchilla ': 'City of Chowchilla, dba: Chowchilla Area Transit ',
 'City of Dinuba ':  'City of Dinuba',
 'Modoc Transportation Agency': 'Modoc Transportation Agency',
 'Butte County Association of Governments/ Butte Regional Transit': 'Butte County Association of Governments',
 'Calaveras County Public Works':  'Calaveras Transit Agency',
 'City of Escalon ':  'City of Escalon, dba: eTrans',
 'County of Mariposa':  'Mariposa County Transit, dba: Mari-Go',
 'County of Shasta Department of Public Works':  'County of Shasta Department of Public Works',
 'County of Siskiyou': 'County of Siskiyou, dba: Siskiyou County Transit',
 'County of Tulare': 'Tulare County Area Transit',
 'Eureka Transit Service':  'City of Eureka, dba: Eureka Transit Service',
 'Kern Regional Transit':  'Kern Regional Transit',
 'Livermore Amador Valley Transit Authority':  'Livermore / Amador Valley Transit Authority',
 'Placer County Public Works (TART & PCT)': 'County of Placer, dba: Placer County Department of Public Works',
 'Plumas County Transportation Commission': 'Plumas County Transportation Commission',
 'San Luis Obispo Regional Transit Authority':  'San Luis Obispo Regional Transit Authority',
 'Sonoma County Transit':  'County of Sonoma, dba: Sonoma County Transit',
 'Sunline Transit Agency':  'SunLine Transit Agency',
 'Tehama County Transit Agency': 'Tehama County',
 'Trinity County Department of Transportation ':  'Trinity County',
 'Tuolumne County Transit Agency (TCTA)':  'Tuolumne County Transit',
 'Amador Transit':  'Amador Regional Transit System',
 'City of Corcoran - Corcoran Area Transit':  'City of Corcoran, dba: Corcoran Area Transit',
 'Yosemite Area Regional Transportation System ':  'Yosemite Area Regional Transportation System',
 'County Connection (Central Contra Costa Transit Authority)': 'Central Contra Costa Transit Authority, dba: COUNTY CONNECTION',
 'Calaveras Transit Agency ': 'Calaveras Transit Agency'}

In [17]:
#failed merges
fail['organization_name'].replace(crosswalk, inplace= True)

/opt/conda/lib/python3.9/site-packages/pandas/core/generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [18]:
#Merging the failed organizations to vehicles 
Test2 = pd.merge(fail, vehicles_gtfs,  how='left', left_on=['organization_name'], right_on=['agency'], indicator=True)

In [19]:
Test2['_merge'].value_counts()

both          248
left_only      55
right_only      0
Name: _merge, dtype: int64

In [20]:
#finding the agencies that are in left only...make sure these are the ones we aren't sure have any matches.
Left_only = Test2[(Test2._merge.str.contains("left_only", case= False))]
#find failed agencies 
Left_only_orgs = Left_only['organization_name'].drop_duplicates().tolist()

In [21]:
#deleting Klamath off since it has GTFS, we only want agencies that do not appear in both GTFS & NTD 
L_only_2_orgs = ['County of Los Angeles - Department of Public Works',
 'County of Nevada Public Works, Transit Services Division',
 'County of Sacramento Department of Transportation',
 'Glenn County Transportation Commission',
 'Stanislaus County Public Works - Transit Division',
 'Alpine County Community Development',
 'Fresno Council of Governments',
 'Greyhound Lines, Inc.']

In [22]:
#appending failed matches to the first data frame
BC_GTFS_NTD = m2.append(Test2, ignore_index=True)

In [23]:
#checking that rows match up
BC_GTFS_NTD.shape

(798, 44)

In [24]:
#checking that organizations are here
BC_GTFS_NTD['organization_name'].nunique()

87

In [25]:
#drop  merge
BC_GTFS_NTD = BC_GTFS_NTD.drop(columns=['_merge'])

### Flag for Black Cat Only

In [26]:
def BC_only(row):
    if row.organization_name in Left_only_orgs:
        return '1'
    else: 
        return '0'  

In [27]:
BC_GTFS_NTD["Is_Agency_In_BC_Only_1_means_Yes"] = BC_GTFS_NTD.apply(lambda x: BC_only(x), axis=1)

In [28]:
#Checking again 
BC_GTFS_NTD[(BC_GTFS_NTD.organization_name.str.contains("greyhound", case= False))].head(1)

,grant_fiscal_year,funding_program,grant_number,project_year,organization_name,upin,description,ali,contract_number,allocationamount,encumbered_amount,expendedamount,activebalance,closedoutbalance,project_status,agency,ntd_id,_0_9,_10_12,_13_15,_16_20,_21_25,_26_30,_31_60,_60plus,total_vehicles,average_age_of_fleet__in_years_,average_lifetime_miles_per_vehicle,sum_15plus,Automobiles,Bus,Other,Service,Train,Van,automobiles_door,bus_doors,train_doors,van_doors,doors_sum,name,itp_id,gtfs_schedule_status,Is_Agency_In_BC_Only_1_means_Yes
578,2017,Section 5311(f),CA-2017-168 | 0017000259-F,2017,"Greyhound Lines, Inc.",BCG0000173,Operating Assistance - Route 1 Service from Ar...,300902,64CO17-00447,161293.0,247887.0,161293.0,0.0,0,Open,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


### Replacing Klamath
* Klamath has GTFS but no NTD records

In [29]:
BC_GTFS_NTD.loc[(BC_GTFS_NTD['organization_name'] == 'Klamath Trinity Non-Emergency Transportation\u200b'), "itp_id"] = "436"
BC_GTFS_NTD.loc[(BC_GTFS_NTD['organization_name'] == 'Klamath Trinity Non-Emergency Transportation\u200b'), "gtfs_schedule_status"] = "needed"

In [30]:
#Checking again 
BC_GTFS_NTD[(BC_GTFS_NTD.organization_name.str.contains("Klamath", case= False))].head(1)

,grant_fiscal_year,funding_program,grant_number,project_year,organization_name,upin,description,ali,contract_number,allocationamount,encumbered_amount,expendedamount,activebalance,closedoutbalance,project_status,agency,ntd_id,_0_9,_10_12,_13_15,_16_20,_21_25,_26_30,_31_60,_60plus,total_vehicles,average_age_of_fleet__in_years_,average_lifetime_miles_per_vehicle,sum_15plus,Automobiles,Bus,Other,Service,Train,Van,automobiles_door,bus_doors,train_doors,van_doors,doors_sum,name,itp_id,gtfs_schedule_status,Is_Agency_In_BC_Only_1_means_Yes
544,2017,5311(f) Cont,TBD,2017,Klamath Trinity Non-Emergency Transportation​,BCG0000168,Operating Assistance,300902,64CO17-00467,8344.0,8344.0,8344.0,0.0,0,Open,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,436,needed,1


### Getting ok/needed/research out of original GTFS data frame

In [31]:
#look at original GTFS status for all the different varities...
BC_GTFS_NTD['gtfs_schedule_status'].value_counts()

ok,needed,needed                                                                                                             173
ok,needed                                                                                                                     87
ok                                                                                                                            81
needed                                                                                                                        78
ok,ok                                                                                                                         28
needed,needed,needed,needed,research,needed                                                                                   28
ok,needed,ok,ok,ok,needed,needed,needed,needed                                                                                24
needed,ok,ok,ok                                                                                  

In [32]:
#create a new column
temp = BC_GTFS_NTD.gtfs_schedule_status.fillna("None")
BC_GTFS_NTD['GTFS_schedule_status_use'] = np.where(temp.str.contains("None"),"None",
                   np.where(temp.str.contains("ok"), "Ok",
                   np.where(temp.str.contains("long"), "Long-term solution needed",
                   np.where(temp.str.contains("research"), "Research", "Needed"))))


In [33]:
BC_GTFS_NTD.sample(3)

,grant_fiscal_year,funding_program,grant_number,project_year,organization_name,upin,description,ali,contract_number,allocationamount,encumbered_amount,expendedamount,activebalance,closedoutbalance,project_status,agency,ntd_id,_0_9,_10_12,_13_15,_16_20,_21_25,_26_30,_31_60,_60plus,total_vehicles,average_age_of_fleet__in_years_,average_lifetime_miles_per_vehicle,sum_15plus,Automobiles,Bus,Other,Service,Train,Van,automobiles_door,bus_doors,train_doors,van_doors,doors_sum,name,itp_id,gtfs_schedule_status,Is_Agency_In_BC_Only_1_means_Yes,GTFS_schedule_status_use
114,2017,Section 5311,CA-2017-168 | 0017000259,2017,City of Ridgecrest,BCG0001068,Operating Assistance Sliding Scale,300902,64BO17-00566,87275.0,87275.0,87275.00,0.00,0,Open,"City of Ridgecrest, dba: Ridgerunner",9R02-91006,8.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,11.0,12.166667,0.0000,NaN,0.0,0.0,0.0,1.0,0.0,10.0,0.0,0.0,0.0,10.0,10.0,City of Ridgecrest,263.0,need long-term solution,0,Long-term solution needed
702,2019,Section 5311(f),CA-2018-114 | 0019000012,2019,Modoc Transportation Agency,BCG0001420,Operating Assistance Sliding Scale - Klamath ...,300902,64CO19-00900,17966.0,17966.0,17653.38,312.62,0,Open,Modoc Transportation Agency,9R02-91008,10.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,14.0,7.750000,0.0000,NaN,0.0,0.0,0.0,0.0,0.0,14.0,0.0,0.0,0.0,14.0,14.0,Modoc Transportation Authority,204.0,"ok,ok",0,Ok
49,2016,Section 5311,CA-2017-025 | 0017000158,2016,Imperial County Transportation Commission,BCG0000186,Operating Assistance,300902,64BO17-00402,164302.0,164302.0,164302.00,0.00,0,Open,Imperial County Transportation Commission,90226,46.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46.0,4.227564,142255.1186,NaN,0.0,16.0,0.0,3.0,0.0,27.0,0.0,32.0,0.0,27.0,59.0,Imperial County Transportation Commission,138.0,"needed,needed,needed,needed",0,Needed


In [34]:
#delete old columns
BC_GTFS_NTD = BC_GTFS_NTD.drop(columns=['simple_GTFS_status','_merge','gtfs_schedule_status','name',])

In [35]:
#get agencies without any data to show up (do this neater...)
#show_up = ['reporter_type','gtfs_status_final','vehicle_type']
#for i in show_up:
    #BC_GTFS_NTD3[i] = BC_GTFS_NTD3[i].fillna('None')

In [36]:
BC_GTFS_NTD.head(1)

,grant_fiscal_year,funding_program,grant_number,project_year,organization_name,upin,description,ali,contract_number,allocationamount,encumbered_amount,expendedamount,activebalance,closedoutbalance,project_status,agency,ntd_id,_0_9,_10_12,_13_15,_16_20,_21_25,_26_30,_31_60,_60plus,total_vehicles,average_age_of_fleet__in_years_,average_lifetime_miles_per_vehicle,sum_15plus,Automobiles,Bus,Other,Service,Train,Van,automobiles_door,bus_doors,train_doors,van_doors,doors_sum,name,itp_id,gtfs_schedule_status,Is_Agency_In_BC_Only_1_means_Yes,GTFS_schedule_status_use
0,2011,Section 5311,CA-18-X047 | 0012000083,2016,Madera County,BCG0000283,Buy <30-Ft Bus For Expansion,111304,64BC17-00408,110663.0,110663.0,101352.02,9310.98,0,Open,"Madera County, dba: Madera County Connection T...",9R02-91005,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0,5.464286,0.0,NaN,0.0,0.0,0.0,0.0,0.0,16.0,0.0,0.0,0.0,16.0,16.0,Madera County,188.0,"ok,needed,needed",0,Ok


In [37]:
BC_GTFS_NTD['itp_id'] = BC_GTFS_NTD['itp_id'].fillna(0)

BC_GTFS_NTD.loc[(BC_GTFS_NTD['itp_id'] == '436'), "itp_id"] = 436

### Parquet

In [39]:
#Adding to GCS
BC_GTFS_NTD.to_parquet("BC_GTFS_NTD.parquet")

# Taking a quick look
* Help, what if an agency has two types of vans...Here I just take the max. If I take the sum, it's not correct...Adds up everything.

In [41]:
Aggregate1 = BC_GTFS_NTD.groupby(['organization_name','reporter_type','GTFS_schedule_status_use', 'Is_Agency_In_BC_Only_1_means_Yes','vehicle_groups',]).agg({'_0_9':'max', '_10_12':'max','_13_15':'max',
       '_16_20':'max', '_21_25':'max', '_26_30':'max', '_31_60':'max', '_60+':'max', 'total_vehicles':'max','average_age_of_fleet__in_years_':'max'
})

KeyError: 'reporter_type'

In [ ]:
Aggregate1

In [ ]:
#reset index
Aggregate1 = Aggregate1.reset_index()

In [ ]:
def bar_chart(df, x_col, y_col):
    y_col_stripped = y_col.replace('_',' ')
    x_col_stripped = x_col.replace('_',' ') 
    chart_title = f"{x_col_stripped} for Black Cat" 
    chart = (alt.Chart(df).mark_bar().encode(
                 x=alt.X(x_col, title=x_col_stripped),
                 y=alt.Y(y_col, title=y_col_stripped),   
                 color =alt.Color(x_col, scale=alt.Scale(range=altair_utils.FIVETHIRTYEIGHT_CATEGORY_COLORS)),
                 tooltip = [alt.Tooltip(x_col_stripped),alt.Tooltip(y_col_stripped)]).interactive().properties(width=600,height=250,
                 title = chart_title)
            )
    display(chart)

## Look at GTFS Status

In [ ]:
Looking_at_GTFS = Aggregate1.groupby(['GTFS_schedule_status_use']).agg({'organization_name':'nunique'})
Looking_at_GTFS = Looking_at_GTFS.reset_index()
Looking_at_GTFS

In [ ]:
bar_chart(Looking_at_GTFS, 'GTFS_schedule_status_use','organization_name')

## Look at Vehicles
* Something weird going on, the total vehicles are wrong for certain counties...
* 5339 is just about buses.

In [ ]:
vehicles = Aggregate1

In [ ]:
vehicles.tail(1)

In [ ]:
#sum up all vehicles except 0-9 category
vehicles['older_than_9']= vehicles['_10_12'] + vehicles['_13_15'] + vehicles['_16_20'] + vehicles['_21_25'] + vehicles['_26_30'] + vehicles['_31_60'] + vehicles['_60+']
#dividing
vehicles['percent_older_than_9'] = (vehicles['older_than_9']/vehicles['total_vehicles'])*100

vehicles.sample(3)

In [ ]:
Vehicles_Age = vehicles.groupby(['organization_name','vehicle_groups']).agg({'percent_older_than_9':'max'})
Vehicles_Age =